In [1]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.3 MB/s eta 0:00:00


In [6]:
"""
MCQ Generator using Gemma or Qwen text generation models.
Designed to run in Google Colab.

Usage:
  - Set MODEL_ID to a Gemma or Qwen model (see options below).
  - Paste your input text into INPUT_TEXT.
  - (Optional) Provide a custom pattern in QUESTION_PATTERN.
  - Run the script and answer each question interactively.
"""

# ──────────────────────────────────────────────
# 1.  INSTALL DEPENDENCIES  (run once in Colab)
# ──────────────────────────────────────────────
# !pip install -q transformers accelerate bitsandbytes

import re
import json
import textwrap
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ──────────────────────────────────────────────
# 2.  CONFIGURATION  – edit these as needed
# ──────────────────────────────────────────────

# --- Model options (pick one) ---
# Qwen  (recommended for Colab free tier – smaller & fast):
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"

# Other options:
# MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"
# MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"       # needs ~16 GB VRAM
# MODEL_ID = "google/gemma-2-2b-it"            # requires HF token
# MODEL_ID = "google/gemma-2-9b-it"            # needs ~20 GB VRAM

# --- Number of questions to generate ---
NUM_QUESTIONS = 5

# ──────────────────────────────────────────────
# 3.  INPUT TEXT  – paste the text you want to
#     turn into MCQs here
# ──────────────────────────────────────────────
INPUT_TEXT = """
Photosynthesis is the process used by plants, algae, and certain bacteria to
harness energy from sunlight and turn it into chemical energy. During
photosynthesis, plants absorb carbon dioxide (CO2) from the air through tiny
pores called stomata and water (H2O) from the soil through their roots.
Using the energy from sunlight, the plant converts these raw materials into
glucose (C6H12O6) and oxygen (O2). The overall chemical equation is:
  6CO2 + 6H2O + light energy → C6H12O6 + 6O2
Photosynthesis occurs mainly in the chloroplasts, where the green pigment
chlorophyll absorbs light energy. The process has two main stages: the
light-dependent reactions (which capture solar energy and produce ATP and
NADPH) and the Calvin cycle (which uses that energy to fix CO2 into glucose).
"""

# ──────────────────────────────────────────────
# 4.  OPTIONAL PATTERN  – describe any extra
#     structure you want the questions to follow.
#     Leave empty ("") for the default style.
# ──────────────────────────────────────────────
QUESTION_PATTERN = """
- Focus on factual recall and conceptual understanding.
- Avoid trivial or trick questions.
- Distractors (wrong options) should be plausible but clearly incorrect.
- The difficulty should be appropriate for a high-school or undergraduate level.
"""

# ──────────────────────────────────────────────
# 5.  HELPER FUNCTIONS
# ──────────────────────────────────────────────

def build_prompt(text: str, pattern: str, n: int) -> str:
    """Build the instruction prompt sent to the model."""
    pattern_block = (
        f"\nAdditional requirements for the questions:\n{pattern.strip()}\n"
        if pattern.strip() else ""
    )

    return textwrap.dedent(f"""
        You are an expert educator. Read the following text and generate exactly
        {n} multiple-choice questions (MCQs) that test understanding of the content.
        {pattern_block}
        Return your answer as a **valid JSON array** with no extra text before or
        after it.  Each element must follow this exact schema:

        {{
          "question": "<question text>",
          "options": {{
            "a": "<option A>",
            "b": "<option B>",
            "c": "<option C>",
            "d": "<option D>"
          }},
          "answer": "<correct letter: a | b | c | d>",
          "explanation": "<brief explanation of why the answer is correct>"
        }}

        ===TEXT START===
        {text.strip()}
        ===TEXT END===

        JSON array:
    """).strip()


def load_model(model_id: str):
    """Load the tokenizer and model with optional 4-bit quantisation."""
    print(f"\n⏳  Loading model: {model_id}  …")

    # Use 4-bit quantisation when a CUDA GPU is available to save VRAM.
    use_cuda = torch.cuda.is_available()
    bnb_config = None
    if use_cuda:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
        )

    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto" if use_cuda else None,
        torch_dtype=torch.bfloat16 if use_cuda else torch.float32,
        trust_remote_code=True,
    )
    model.eval()
    print("✅  Model loaded.\n")
    return tokenizer, model


def generate_mcqs(tokenizer, model, prompt: str, max_new_tokens: int = 2048) -> str:
    """Run inference and return the raw model output string."""
    device = next(model.parameters()).device
    messages = [{"role": "user", "content": prompt}]

    # --- tokenise input -------------------------------------------------------
    # apply_chat_template may return a plain Tensor OR a BatchEncoding object
    # (which is NOT a plain dict, so isinstance(x, dict) misses it).
    # Safest approach: always ask for a list of ints, then convert manually.
    try:
        token_list = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors=None,      # returns a plain Python list of ints
        )
        input_ids = torch.tensor([token_list], dtype=torch.long).to(device)
    except Exception:
        # Fallback: plain tokenisation (no chat template)
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    prompt_len = input_ids.shape[-1]               # save BEFORE generation

    # --- generate -------------------------------------------------------------
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=False,          # greedy → deterministic JSON
            temperature=1.0,
            pad_token_id=tokenizer.eos_token_id,
        )

    # Decode only the newly generated tokens (skip the echoed prompt)
    new_tokens = output_ids[0][prompt_len:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True)


def extract_json(raw: str) -> list:
    """
    Extract and parse the first JSON array found in the model output.
    Falls back to a regex search if the output contains extra prose.
    """
    # Try direct parse first
    raw = raw.strip()
    try:
        data = json.loads(raw)
        if isinstance(data, list):
            return data
    except json.JSONDecodeError:
        pass

    # Search for the first [...] block in the output
    match = re.search(r"\[.*?\]", raw, re.DOTALL)
    if match:
        try:
            data = json.loads(match.group())
            if isinstance(data, list):
                return data
        except json.JSONDecodeError:
            pass

    raise ValueError(
        "Could not parse a JSON array from the model output.\n"
        f"Raw output (first 800 chars):\n{raw[:800]}"
    )


def validate_mcq(mcq: dict, idx: int) -> bool:
    """Basic sanity check for a single MCQ dict."""
    required_keys = {"question", "options", "answer", "explanation"}
    if not required_keys.issubset(mcq.keys()):
        print(f"  ⚠️  Question {idx+1} is missing keys – skipping.")
        return False
    if not isinstance(mcq["options"], dict) or not {"a","b","c","d"}.issubset(mcq["options"]):
        print(f"  ⚠️  Question {idx+1} has malformed options – skipping.")
        return False
    if mcq["answer"].lower() not in {"a", "b", "c", "d"}:
        print(f"  ⚠️  Question {idx+1} has an invalid answer key – skipping.")
        return False
    return True


# ──────────────────────────────────────────────
# 6.  QUIZ RUNNER
# ──────────────────────────────────────────────

def print_separator(char="─", width=60):
    print(char * width)


def run_quiz(mcqs: list):
    """Present each MCQ to the user and evaluate their answers."""
    print_separator("═")
    print("🎓  QUIZ TIME!  Answer each question by typing a, b, c, or d.")
    print_separator("═")

    score = 0
    total = len(mcqs)

    for i, mcq in enumerate(mcqs):
        print(f"\nQ{i+1} / {total}: {mcq['question']}\n")
        for letter in ["a", "b", "c", "d"]:
            print(f"  {letter})  {mcq['options'][letter]}")

        # Get a valid answer from the user
        while True:
            raw = input("\nYour answer (a/b/c/d): ").strip().lower()
            if raw in {"a", "b", "c", "d"}:
                break
            print("  Please type a, b, c, or d.")

        correct = mcq["answer"].lower()

        if raw == correct:
            score += 1
            print("\n✅  Correct!\n")
        else:
            print(f"\n❌  Wrong!  The correct answer was ({correct}) "
                  f"{mcq['options'][correct]}")
            print(f"\n💡  Explanation: {mcq['explanation']}\n")

        print_separator()

    # Final score
    print(f"\n🏆  Quiz complete!  You scored {score} / {total}.")
    pct = round(score / total * 100)
    if pct == 100:
        print("    Perfect score – outstanding! 🎉")
    elif pct >= 80:
        print("    Great job! 👍")
    elif pct >= 60:
        print("    Good effort – review the explanations above to improve.")
    else:
        print("    Keep studying – you'll get there! 📚")


# ──────────────────────────────────────────────
# 7.  MAIN
# ──────────────────────────────────────────────

def main():
    # Step 1: Load model
    tokenizer, model = load_model(MODEL_ID)

    # Step 2: Build the prompt
    prompt = build_prompt(INPUT_TEXT, QUESTION_PATTERN, NUM_QUESTIONS)

    # Step 3: Generate MCQs
    print("⚙️   Generating questions …\n")
    raw_output = generate_mcqs(tokenizer, model, prompt)

    # Step 4: Parse the JSON
    try:
        mcqs_raw = extract_json(raw_output)
    except ValueError as e:
        print(f"\n❌  Error parsing model output:\n{e}")
        return

    # Step 5: Validate and filter
    mcqs = [mcq for i, mcq in enumerate(mcqs_raw) if validate_mcq(mcq, i)]

    if not mcqs:
        print("❌  No valid questions were generated. "
              "Try a different model or adjust your input text.")
        return

    print(f"✅  {len(mcqs)} question(s) generated successfully.\n")

    # Step 6: Run the interactive quiz
    run_quiz(mcqs)


if __name__ == "__main__":
    main()


⏳  Loading model: Qwen/Qwen2.5-1.5B-Instruct  …


Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


✅  Model loaded.

⚙️   Generating questions …

✅  5 question(s) generated successfully.

════════════════════════════════════════════════════════════
🎓  QUIZ TIME!  Answer each question by typing a, b, c, or d.
════════════════════════════════════════════════════════════

Q1 / 5: What is the primary function of photosynthesis?

  a)  To convert light energy into electrical energy
  b)  To provide food for animals
  c)  To release oxygen into the atmosphere
  d)  To store large amounts of water

Your answer (a/b/c/d): a

✅  Correct!

────────────────────────────────────────────────────────────

Q2 / 5: In which part of the plant does photosynthesis occur?

  a)  Leaves
  b)  Stem
  c)  Root
  d)  All parts equally

Your answer (a/b/c/d): a

✅  Correct!

────────────────────────────────────────────────────────────

Q3 / 5: How many molecules of CO2 are required for one molecule of glucose during photosynthesis?

  a)  3
  b)  4
  c)  5
  d)  6

Your answer (a/b/c/d): 6
  Please type a, b

In [7]:
"""
MCQ Generator using Gemma or Qwen text generation models.
Designed to run in Google Colab.

Usage:
  - Set MODEL_ID to a Gemma or Qwen model (see options below).
  - Paste your input text into INPUT_TEXT.
  - (Optional) Provide a custom pattern in QUESTION_PATTERN.
  - Run the script and answer each question interactively.
"""

# ──────────────────────────────────────────────
# 1.  INSTALL DEPENDENCIES  (run once in Colab)
# ──────────────────────────────────────────────
# !pip install -q transformers accelerate bitsandbytes

import re
import json
import textwrap
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ──────────────────────────────────────────────
# 2.  CONFIGURATION  – edit these as needed
# ──────────────────────────────────────────────

# --- Model options (pick one) ---
# Qwen  (recommended for Colab free tier – smaller & fast):
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"

# Other options:
# MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"
# MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"       # needs ~16 GB VRAM
# MODEL_ID = "google/gemma-2-2b-it"            # requires HF token
# MODEL_ID = "google/gemma-2-9b-it"            # needs ~20 GB VRAM

# --- Number of questions to generate ---
NUM_QUESTIONS = 5

# ──────────────────────────────────────────────
# 3.  INPUT TEXT  – paste the text you want to
#     turn into MCQs here
# ──────────────────────────────────────────────
INPUT_TEXT = """
Photosynthesis is the process used by plants, algae, and certain bacteria to
harness energy from sunlight and turn it into chemical energy. During
photosynthesis, plants absorb carbon dioxide (CO2) from the air through tiny
pores called stomata and water (H2O) from the soil through their roots.
Using the energy from sunlight, the plant converts these raw materials into
glucose (C6H12O6) and oxygen (O2). The overall chemical equation is:
  6CO2 + 6H2O + light energy → C6H12O6 + 6O2
Photosynthesis occurs mainly in the chloroplasts, where the green pigment
chlorophyll absorbs light energy. The process has two main stages: the
light-dependent reactions (which capture solar energy and produce ATP and
NADPH) and the Calvin cycle (which uses that energy to fix CO2 into glucose).
"""

# ──────────────────────────────────────────────
# 4.  OPTIONAL PATTERN  – describe any extra
#     structure you want the questions to follow.
#     Leave empty ("") for the default style.
# ──────────────────────────────────────────────
QUESTION_PATTERN = """
- Focus on factual recall and conceptual understanding.
- Avoid trivial or trick questions.
- Distractors (wrong options) should be plausible but clearly incorrect.
- The difficulty should be appropriate for a high-school or undergraduate level.
"""

# ──────────────────────────────────────────────
# 5.  HELPER FUNCTIONS
# ──────────────────────────────────────────────

def build_prompt(text: str, pattern: str, n: int) -> str:
    """Build the instruction prompt sent to the model."""
    pattern_block = (
        f"\nAdditional requirements for the questions:\n{pattern.strip()}\n"
        if pattern.strip() else ""
    )

    return textwrap.dedent(f"""
        You are an expert educator. Read the following text and generate exactly
        {n} multiple-choice questions (MCQs) that test understanding of the content.
        {pattern_block}
        Return your answer as a **valid JSON array** with no extra text before or
        after it.  Each element must follow this exact schema:

        {{
          "question": "<question text>",
          "options": {{
            "a": "<option A>",
            "b": "<option B>",
            "c": "<option C>",
            "d": "<option D>"
          }},
          "answer": "<correct letter: a | b | c | d>",
          "explanation": "<brief explanation of why the answer is correct>"
        }}

        ===TEXT START===
        {text.strip()}
        ===TEXT END===

        JSON array:
    """).strip()


def load_model(model_id: str):
    """Load the tokenizer and model with optional 4-bit quantisation."""
    print(f"\n⏳  Loading model: {model_id}  …")

    # Use 4-bit quantisation when a CUDA GPU is available to save VRAM.
    use_cuda = torch.cuda.is_available()
    bnb_config = None
    if use_cuda:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
        )

    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    # Qwen's pad token == eos token by default, which triggers a harmless but
    # noisy warning.  Adding a dedicated pad token silences it.
    if tokenizer.pad_token_id is None or tokenizer.pad_token_id == tokenizer.eos_token_id:
        tokenizer.add_special_tokens({"pad_token": "<|pad|>"})
        needs_resize = True
    else:
        needs_resize = False

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto" if use_cuda else None,
        torch_dtype=torch.bfloat16 if use_cuda else torch.float32,
        trust_remote_code=True,
    )
    model.eval()
    if needs_resize:
        model.resize_token_embeddings(len(tokenizer))
    print("✅  Model loaded.\n")
    return tokenizer, model


def generate_mcqs(tokenizer, model, prompt: str, max_new_tokens: int = 2048) -> str:
    """Run inference and return the raw model output string."""
    device = next(model.parameters()).device
    messages = [{"role": "user", "content": prompt}]

    # --- tokenise input -------------------------------------------------------
    # apply_chat_template may return a plain Tensor OR a BatchEncoding object
    # (which is NOT a plain dict, so isinstance(x, dict) misses it).
    # Safest approach: always ask for a list of ints, then convert manually.
    try:
        token_list = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors=None,      # returns a plain Python list of ints
        )
        input_ids = torch.tensor([token_list], dtype=torch.long).to(device)
    except Exception:
        # Fallback: plain tokenisation (no chat template)
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    prompt_len = input_ids.shape[-1]               # save BEFORE generation
    attention_mask = torch.ones_like(input_ids)    # all tokens are real (no padding)

    # --- generate -------------------------------------------------------------
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=False,          # greedy → deterministic JSON
            temperature=1.0,
            pad_token_id=tokenizer.pad_token_id,
        )

    # Decode only the newly generated tokens (skip the echoed prompt)
    new_tokens = output_ids[0][prompt_len:]
    return tokenizer.decode(new_tokens, skip_special_tokens=True)


def extract_json(raw: str) -> list:
    """
    Extract and parse the first JSON array found in the model output.
    Falls back to a regex search if the output contains extra prose.
    """
    # Try direct parse first
    raw = raw.strip()
    try:
        data = json.loads(raw)
        if isinstance(data, list):
            return data
    except json.JSONDecodeError:
        pass

    # Search for the first [...] block in the output
    match = re.search(r"\[.*?\]", raw, re.DOTALL)
    if match:
        try:
            data = json.loads(match.group())
            if isinstance(data, list):
                return data
        except json.JSONDecodeError:
            pass

    raise ValueError(
        "Could not parse a JSON array from the model output.\n"
        f"Raw output (first 800 chars):\n{raw[:800]}"
    )


def validate_mcq(mcq: dict, idx: int) -> bool:
    """Basic sanity check for a single MCQ dict."""
    required_keys = {"question", "options", "answer", "explanation"}
    if not required_keys.issubset(mcq.keys()):
        print(f"  ⚠️  Question {idx+1} is missing keys – skipping.")
        return False
    if not isinstance(mcq["options"], dict) or not {"a","b","c","d"}.issubset(mcq["options"]):
        print(f"  ⚠️  Question {idx+1} has malformed options – skipping.")
        return False
    if mcq["answer"].lower() not in {"a", "b", "c", "d"}:
        print(f"  ⚠️  Question {idx+1} has an invalid answer key – skipping.")
        return False
    return True


# ──────────────────────────────────────────────
# 6.  QUIZ RUNNER
# ──────────────────────────────────────────────

def print_separator(char="─", width=60):
    print(char * width)


def run_quiz(mcqs: list):
    """Present each MCQ to the user and evaluate their answers."""
    print_separator("═")
    print("🎓  QUIZ TIME!  Answer each question by typing a, b, c, or d.")
    print_separator("═")

    score = 0
    total = len(mcqs)

    for i, mcq in enumerate(mcqs):
        print(f"\nQ{i+1} / {total}: {mcq['question']}\n")
        for letter in ["a", "b", "c", "d"]:
            print(f"  {letter})  {mcq['options'][letter]}")

        # Get a valid answer from the user
        while True:
            raw = input("\nYour answer (a/b/c/d): ").strip().lower()
            if raw in {"a", "b", "c", "d"}:
                break
            print("  Please type a, b, c, or d.")

        correct = mcq["answer"].lower()

        if raw == correct:
            score += 1
            print("\n✅  Correct!\n")
        else:
            print(f"\n❌  Wrong!  The correct answer was ({correct}) "
                  f"{mcq['options'][correct]}")
            print(f"\n💡  Explanation: {mcq['explanation']}\n")

        print_separator()

    # Final score
    print(f"\n🏆  Quiz complete!  You scored {score} / {total}.")
    pct = round(score / total * 100)
    if pct == 100:
        print("    Perfect score – outstanding! 🎉")
    elif pct >= 80:
        print("    Great job! 👍")
    elif pct >= 60:
        print("    Good effort – review the explanations above to improve.")
    else:
        print("    Keep studying – you'll get there! 📚")


# ──────────────────────────────────────────────
# 7.  MAIN
# ──────────────────────────────────────────────

def main():
    # Step 1: Load model
    tokenizer, model = load_model(MODEL_ID)

    # Step 2: Build the prompt
    prompt = build_prompt(INPUT_TEXT, QUESTION_PATTERN, NUM_QUESTIONS)

    # Step 3: Generate MCQs
    print("⚙️   Generating questions …\n")
    raw_output = generate_mcqs(tokenizer, model, prompt)

    # Step 4: Parse the JSON
    try:
        mcqs_raw = extract_json(raw_output)
    except ValueError as e:
        print(f"\n❌  Error parsing model output:\n{e}")
        return

    # Step 5: Validate and filter
    mcqs = [mcq for i, mcq in enumerate(mcqs_raw) if validate_mcq(mcq, i)]

    if not mcqs:
        print("❌  No valid questions were generated. "
              "Try a different model or adjust your input text.")
        return

    print(f"✅  {len(mcqs)} question(s) generated successfully.\n")

    # Step 6: Run the interactive quiz
    run_quiz(mcqs)


if __name__ == "__main__":
    main()


⏳  Loading model: Qwen/Qwen2.5-1.5B-Instruct  …


Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

✅  Model loaded.

⚙️   Generating questions …

✅  5 question(s) generated successfully.

════════════════════════════════════════════════════════════
🎓  QUIZ TIME!  Answer each question by typing a, b, c, or d.
════════════════════════════════════════════════════════════

Q1 / 5: What is the primary function of photosynthesis?

  a)  To convert light energy into electrical energy
  b)  To provide food for animals
  c)  To release oxygen into the atmosphere
  d)  To store large amounts of water

Your answer (a/b/c/d): b

❌  Wrong!  The correct answer was (a) To convert light energy into electrical energy

💡  Explanation: Photosynthesis primarily converts light energy into chemical energy stored in glucose.

────────────────────────────────────────────────────────────

Q2 / 5: In which part of the plant does photosynthesis occur?

  a)  Leaves
  b)  Stem
  c)  Root
  d)  All parts equally

Your answer (a/b/c/d): a

✅  Correct!

────────────────────────────────────────────────────────────

In [8]:
# Cell 1 – install
!pip install -q google-generativeai

In [9]:
"""
MCQ Generator using Google Gemini API.
Designed to run in Google Colab.

Colab setup (run these cells first):
──────────────────────────────────────
Cell 1 – install the SDK:
  !pip install -q google-generativeai

Cell 2 – set your API key:
  from google.colab import userdata
  # Add your key in Colab → Secrets (🔑 icon) as  GEMINI_API_KEY
  # OR just paste it directly into GEMINI_API_KEY below (not recommended for sharing)

Cell 3 – run this script:
  !python mcq_generator_gemini.py

Get a free API key at: https://aistudio.google.com/app/apikey
──────────────────────────────────────
"""

import re
import json
import textwrap
import os

try:
    import google.generativeai as genai
except ImportError:
    raise SystemExit("❌  Run:  !pip install -q google-generativeai  then restart.")

# ──────────────────────────────────────────────
# CONFIGURATION
# ──────────────────────────────────────────────

# Paste your key here, OR store it in Colab Secrets as GEMINI_API_KEY
GEMINI_API_KEY = ""

# Model options:
#   "gemini-2.0-flash"          ← default: fast & free tier friendly
#   "gemini-2.0-flash-lite"     ← cheapest / fastest
#   "gemini-1.5-pro"            ← most capable
MODEL = "gemini-2.0-flash"

NUM_QUESTIONS = 5

# ──────────────────────────────────────────────
# INPUT TEXT
# ──────────────────────────────────────────────
INPUT_TEXT = """
Photosynthesis is the process used by plants, algae, and certain bacteria to
harness energy from sunlight and turn it into chemical energy. During
photosynthesis, plants absorb carbon dioxide (CO2) from the air through tiny
pores called stomata and water (H2O) from the soil through their roots.
Using the energy from sunlight, the plant converts these raw materials into
glucose (C6H12O6) and oxygen (O2). The overall chemical equation is:
  6CO2 + 6H2O + light energy → C6H12O6 + 6O2
Photosynthesis occurs mainly in the chloroplasts, where the green pigment
chlorophyll absorbs light energy. The process has two main stages: the
light-dependent reactions (which capture solar energy and produce ATP and
NADPH) and the Calvin cycle (which uses that energy to fix CO2 into glucose).
"""

# ──────────────────────────────────────────────
# OPTIONAL PATTERN  – leave empty "" to skip
# ──────────────────────────────────────────────
QUESTION_PATTERN = """
- Focus on factual recall and conceptual understanding.
- Avoid trivial or trick questions.
- Distractors (wrong options) should be plausible but clearly incorrect.
- Difficulty: high-school or undergraduate level.
"""

# ──────────────────────────────────────────────
# HELPERS
# ──────────────────────────────────────────────

def resolve_api_key() -> str:
    """Try Colab Secrets → env var → hardcoded constant."""
    if GEMINI_API_KEY:
        return GEMINI_API_KEY
    # Colab Secrets
    try:
        from google.colab import userdata
        key = userdata.get("GEMINI_API_KEY")
        if key:
            return key
    except Exception:
        pass
    # Environment variable
    key = os.environ.get("GEMINI_API_KEY", "")
    if key:
        return key
    raise SystemExit(
        "❌  No API key found.\n"
        "    Options:\n"
        "    1) Add GEMINI_API_KEY in Colab Secrets (🔑 icon on the left sidebar)\n"
        "    2) Set GEMINI_API_KEY = 'your-key' at the top of this script\n"
        "    Get a free key at: https://aistudio.google.com/app/apikey"
    )


def build_prompt(text: str, pattern: str, n: int) -> str:
    pattern_block = (
        f"\nAdditional requirements:\n{pattern.strip()}\n"
        if pattern.strip() else ""
    )
    return textwrap.dedent(f"""
        You are an expert educator. Read the text below and generate exactly
        {n} multiple-choice questions (MCQs) that test understanding of it.
        {pattern_block}
        Return ONLY a valid JSON array – no markdown fences, no extra text.
        Each element must follow this exact schema:
        {{
          "question": "<question text>",
          "options": {{"a": "...", "b": "...", "c": "...", "d": "..."}},
          "answer": "<a | b | c | d>",
          "explanation": "<why the answer is correct>"
        }}

        ===TEXT===
        {text.strip()}
        ===END===
    """).strip()


def generate_mcqs(model, prompt: str) -> str:
    """Call the Gemini API and return raw text."""
    response = model.generate_content(
        prompt,
        generation_config=genai.GenerationConfig(
            temperature=0,          # deterministic → clean JSON
            max_output_tokens=2048,
        ),
    )
    return response.text


def extract_json(raw: str) -> list:
    """Parse the first JSON array from the model output."""
    raw = raw.strip()
    # Strip markdown fences if the model adds them anyway
    raw = re.sub(r"^```(?:json)?\s*", "", raw)
    raw = re.sub(r"\s*```$", "", raw)

    try:
        data = json.loads(raw)
        if isinstance(data, list):
            return data
    except json.JSONDecodeError:
        pass

    match = re.search(r"\[.*\]", raw, re.DOTALL)
    if match:
        try:
            data = json.loads(match.group())
            if isinstance(data, list):
                return data
        except json.JSONDecodeError:
            pass

    raise ValueError(
        "Could not parse a JSON array from the model output.\n"
        f"Raw output (first 800 chars):\n{raw[:800]}"
    )


def validate_mcq(mcq: dict, idx: int) -> bool:
    required = {"question", "options", "answer", "explanation"}
    if not required.issubset(mcq):
        print(f"  ⚠️  Q{idx+1} missing keys – skipping.")
        return False
    if not isinstance(mcq["options"], dict) or not {"a","b","c","d"}.issubset(mcq["options"]):
        print(f"  ⚠️  Q{idx+1} malformed options – skipping.")
        return False
    if mcq["answer"].lower() not in {"a","b","c","d"}:
        print(f"  ⚠️  Q{idx+1} invalid answer key – skipping.")
        return False
    return True


# ──────────────────────────────────────────────
# QUIZ RUNNER
# ──────────────────────────────────────────────

def sep(char="─", w=60):
    print(char * w)


def run_quiz(mcqs: list):
    sep("═")
    print("🎓  QUIZ TIME!  Type a, b, c, or d to answer each question.")
    sep("═")

    score = 0
    for i, mcq in enumerate(mcqs):
        print(f"\nQ{i+1}/{len(mcqs)}: {mcq['question']}\n")
        for letter in "abcd":
            print(f"  {letter})  {mcq['options'][letter]}")

        while True:
            ans = input("\nYour answer (a/b/c/d): ").strip().lower()
            if ans in {"a","b","c","d"}:
                break
            print("  Please type a, b, c, or d.")

        correct = mcq["answer"].lower()
        if ans == correct:
            score += 1
            print("\n✅  Correct!\n")
        else:
            print(f"\n❌  Wrong!  Correct answer: ({correct}) {mcq['options'][correct]}")
            print(f"💡  {mcq['explanation']}\n")
        sep()

    print(f"\n🏆  Score: {score}/{len(mcqs)}")
    pct = score / len(mcqs) * 100
    if pct == 100:  print("    Perfect! 🎉")
    elif pct >= 80: print("    Great job! 👍")
    elif pct >= 60: print("    Good effort – review the explanations above.")
    else:           print("    Keep studying! 📚")


# ──────────────────────────────────────────────
# MAIN
# ──────────────────────────────────────────────

def main():
    # 1. Auth
    api_key = resolve_api_key()
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(MODEL)
    print(f"🤖  Model  : {MODEL}")
    print(f"📋  Generating {NUM_QUESTIONS} questions …\n")

    # 2. Generate
    prompt = build_prompt(INPUT_TEXT, QUESTION_PATTERN, NUM_QUESTIONS)
    try:
        raw = generate_mcqs(model, prompt)
    except Exception as e:
        print(f"❌  Gemini API error: {e}")
        return

    # 3. Parse
    try:
        mcqs_raw = extract_json(raw)
    except ValueError as e:
        print(f"❌  {e}")
        return

    # 4. Validate
    mcqs = [m for i, m in enumerate(mcqs_raw) if validate_mcq(m, i)]
    if not mcqs:
        print("❌  No valid questions generated.")
        return

    print(f"✅  {len(mcqs)} question(s) ready.\n")

    # 5. Quiz
    run_quiz(mcqs)


if __name__ == "__main__":
    main()

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


🤖  Model  : gemini-2.0-flash
📋  Generating 5 questions …



❌  Gemini API error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 0, model: gemini-2.0-flash
Please retry in 27.020577868s.


In [11]:
"""
MCQ Generator using Google Gemini API (new google-genai SDK).
Designed to run in Google Colab.

Colab setup:
──────────────────────────────────────
Cell 1 – install the NEW SDK:
  !pip install -q google-genai

Cell 2 – add your key in Colab Secrets (🔑 left sidebar)
  Name: GEMINI_API_KEY

Cell 3 – run:
  !python mcq_generator_gemini.py

Free key → https://aistudio.google.com/app/apikey
──────────────────────────────────────
"""

import re
import json
import textwrap
import os
import time

try:
    from google import genai
    from google.genai import types
except ImportError:
    raise SystemExit("❌  Run:  !pip install -q google-genai  then restart the runtime.")

# ──────────────────────────────────────────────
# CONFIGURATION
# ──────────────────────────────────────────────

# Paste your key here OR store it in Colab Secrets as GEMINI_API_KEY
GEMINI_API_KEY = ""

# Model options (free tier):
#   "gemini-2.0-flash-lite"   ← most generous free quota  ✅ recommended
#   "gemini-2.0-flash"        ← fast, slightly lower free quota
#   "gemini-1.5-flash"        ← older but very stable
MODEL = "gemini-2.0-flash-lite"

NUM_QUESTIONS = 5

# Retry settings for 429 / quota errors
MAX_RETRIES = 4          # number of retry attempts
RETRY_WAIT  = 30         # seconds to wait between retries (if no hint in error)

# ──────────────────────────────────────────────
# INPUT TEXT
# ──────────────────────────────────────────────
INPUT_TEXT = """
Photosynthesis is the process used by plants, algae, and certain bacteria to
harness energy from sunlight and turn it into chemical energy. During
photosynthesis, plants absorb carbon dioxide (CO2) from the air through tiny
pores called stomata and water (H2O) from the soil through their roots.
Using the energy from sunlight, the plant converts these raw materials into
glucose (C6H12O6) and oxygen (O2). The overall chemical equation is:
  6CO2 + 6H2O + light energy → C6H12O6 + 6O2
Photosynthesis occurs mainly in the chloroplasts, where the green pigment
chlorophyll absorbs light energy. The process has two main stages: the
light-dependent reactions (which capture solar energy and produce ATP and
NADPH) and the Calvin cycle (which uses that energy to fix CO2 into glucose).
"""

# ──────────────────────────────────────────────
# OPTIONAL PATTERN  – leave "" to skip
# ──────────────────────────────────────────────
QUESTION_PATTERN = """
- Focus on factual recall and conceptual understanding.
- Avoid trivial or trick questions.
- Distractors (wrong options) should be plausible but clearly incorrect.
- Difficulty: high-school or undergraduate level.
"""

# ──────────────────────────────────────────────
# HELPERS
# ──────────────────────────────────────────────

def resolve_api_key() -> str:
    """Try hardcoded constant → Colab Secrets → environment variable."""
    if GEMINI_API_KEY:
        return GEMINI_API_KEY
    try:
        from google.colab import userdata
        key = userdata.get("GEMINI_API_KEY")
        if key:
            return key
    except Exception:
        pass
    key = os.environ.get("GEMINI_API_KEY", "")
    if key:
        return key
    raise SystemExit(
        "❌  No API key found.\n"
        "    1) Add GEMINI_API_KEY in Colab Secrets (🔑 left sidebar)\n"
        "    2) OR paste it into GEMINI_API_KEY at the top of this script.\n"
        "    Free key → https://aistudio.google.com/app/apikey"
    )


def build_prompt(text: str, pattern: str, n: int) -> str:
    pattern_block = (
        f"\nAdditional requirements:\n{pattern.strip()}\n"
        if pattern.strip() else ""
    )
    return textwrap.dedent(f"""
        You are an expert educator. Read the text below and generate exactly
        {n} multiple-choice questions (MCQs) that test understanding of it.
        {pattern_block}
        Return ONLY a valid JSON array – no markdown fences, no extra text.
        Each element must follow this exact schema:
        {{
          "question": "<question text>",
          "options": {{"a": "...", "b": "...", "c": "...", "d": "..."}},
          "answer": "<a | b | c | d>",
          "explanation": "<why the answer is correct>"
        }}

        ===TEXT===
        {text.strip()}
        ===END===
    """).strip()


def call_gemini(client: genai.Client, prompt: str) -> str:
    """
    Call the Gemini API with automatic retry on 429 quota errors.
    Parses the retry-delay hint from the error message when available.
    """
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            response = client.models.generate_content(
                model=MODEL,
                contents=prompt,
                config=types.GenerateContentConfig(
                    temperature=0,
                    max_output_tokens=2048,
                ),
            )
            return response.text

        except Exception as e:
            err = str(e)
            is_quota = "429" in err or "quota" in err.lower() or "rate" in err.lower()

            if is_quota and attempt < MAX_RETRIES:
                # Read the suggested wait time from the error if present
                wait = RETRY_WAIT
                m = re.search(r"retry in ([\d.]+)s", err, re.IGNORECASE)
                if m:
                    wait = int(float(m.group(1))) + 2   # small buffer
                print(f"  ⏳  Quota limit hit – waiting {wait}s "
                      f"(attempt {attempt}/{MAX_RETRIES}) …")
                time.sleep(wait)
            else:
                raise   # non-quota error or retries exhausted


def extract_json(raw: str) -> list:
    raw = raw.strip()
    # Strip markdown fences the model might add despite instructions
    raw = re.sub(r"^```(?:json)?\s*", "", raw)
    raw = re.sub(r"\s*```$", "", raw)

    try:
        data = json.loads(raw)
        if isinstance(data, list):
            return data
    except json.JSONDecodeError:
        pass

    # Fallback: find the first [...] block
    match = re.search(r"\[.*\]", raw, re.DOTALL)
    if match:
        try:
            data = json.loads(match.group())
            if isinstance(data, list):
                return data
        except json.JSONDecodeError:
            pass

    raise ValueError(
        "Could not parse a JSON array from the model output.\n"
        f"Raw output (first 800 chars):\n{raw[:800]}"
    )


def validate_mcq(mcq: dict, idx: int) -> bool:
    required = {"question", "options", "answer", "explanation"}
    if not required.issubset(mcq):
        print(f"  ⚠️  Q{idx+1} missing keys – skipping.")
        return False
    if not isinstance(mcq["options"], dict) or not {"a","b","c","d"}.issubset(mcq["options"]):
        print(f"  ⚠️  Q{idx+1} malformed options – skipping.")
        return False
    if mcq["answer"].lower() not in {"a","b","c","d"}:
        print(f"  ⚠️  Q{idx+1} invalid answer key – skipping.")
        return False
    return True


# ──────────────────────────────────────────────
# QUIZ RUNNER
# ──────────────────────────────────────────────

def sep(char="─", w=60):
    print(char * w)


def run_quiz(mcqs: list):
    sep("═")
    print("🎓  QUIZ TIME!  Type a, b, c, or d to answer each question.")
    sep("═")

    score = 0
    for i, mcq in enumerate(mcqs):
        print(f"\nQ{i+1}/{len(mcqs)}: {mcq['question']}\n")
        for letter in "abcd":
            print(f"  {letter})  {mcq['options'][letter]}")

        while True:
            ans = input("\nYour answer (a/b/c/d): ").strip().lower()
            if ans in {"a","b","c","d"}:
                break
            print("  Please type a, b, c, or d.")

        correct = mcq["answer"].lower()
        if ans == correct:
            score += 1
            print("\n✅  Correct!\n")
        else:
            print(f"\n❌  Wrong!  Correct answer: ({correct}) {mcq['options'][correct]}")
            print(f"💡  {mcq['explanation']}\n")
        sep()

    print(f"\n🏆  Score: {score}/{len(mcqs)}")
    pct = score / len(mcqs) * 100
    if pct == 100:  print("    Perfect! 🎉")
    elif pct >= 80: print("    Great job! 👍")
    elif pct >= 60: print("    Good effort – review the explanations above.")
    else:           print("    Keep studying! 📚")


# ──────────────────────────────────────────────
# MAIN
# ──────────────────────────────────────────────

def main():
    api_key = resolve_api_key()
    client  = genai.Client(api_key=api_key)

    print(f"🤖  Model  : {MODEL}")
    print(f"📋  Generating {NUM_QUESTIONS} questions …\n")

    prompt = build_prompt(INPUT_TEXT, QUESTION_PATTERN, NUM_QUESTIONS)

    try:
        raw = call_gemini(client, prompt)
    except Exception as e:
        print(f"❌  Gemini API error: {e}")
        return

    try:
        mcqs_raw = extract_json(raw)
    except ValueError as e:
        print(f"❌  {e}")
        return

    mcqs = [m for i, m in enumerate(mcqs_raw) if validate_mcq(m, i)]
    if not mcqs:
        print("❌  No valid questions generated.")
        return

    print(f"✅  {len(mcqs)} question(s) ready.\n")
    run_quiz(mcqs)


if __name__ == "__main__":
    main()


🤖  Model  : gemini-2.0-flash-lite
📋  Generating 5 questions …

  ⏳  Quota limit hit – waiting 20s (attempt 1/4) …
  ⏳  Quota limit hit – waiting 60s (attempt 2/4) …


KeyboardInterrupt: 